In [1]:
import re
import csv
import pickle
import pandas as pd
import numpy as np

In [2]:
#start processing data twitter
def processTweet(tweet):
    # process the tweets
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
#     print(tweet)
#     print(' ')
    return tweet
#end

In [3]:
#initialize stopWords
stopWords = []

In [4]:
#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

In [5]:
#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords
#end

In [6]:
#start getfeatureVector
def getFeatureVector(tweet):
    featureVector = []
    # tweet=stemmer.stem(tweet)
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
           featureVector.append(w.lower())
#     print(featureVector)
#     print(' ')
    return featureVector
#end

In [7]:
#start replace singkatan/kata alay
def translator(tweet):
    j = 0
    for kata in tweet:
        with open('data/singkatan.txt', 'r') as csvfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(csvfile, delimiter="=")
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if kata == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    tweet[j] = row[1]
            csvfile.close()
        j = j + 1
#     print(' '.join(tweet))
#     print('')
    return tweet    
#end process

In [8]:
#start pos tagging process
def posTagging(tweet):
    temp = []
    dataFromFile = pd.read_csv('data/kamus.csv')
    dataFromFile.columns = ['kata', 'postag', 'sentimen']
    dataFromFile['sentimen'] = dataFromFile['sentimen'].replace({'Positif': True, 'Negatif': False})
    for kata in tweet:
        if kata in dataFromFile.kata.tolist():
            row = dataFromFile.loc[dataFromFile.kata == kata].values[0]
            temp.append(row.tolist())
        else:
            temp.append([kata, 'unknown', 0])

    print(temp)
    print(' ')
    return(temp)
#end process

In [9]:
def RuleDes(tweet):
    term1= "Verba"
    term2="Adjektiva"
    term3="Verba-di"
    term4="Nomina"
    term5="Preposisi"
    term6="unknown"
    temp_sentimen = []
    hasil_rulebox = []
    for row in tweet:
        rulebox = ['','','','','','']
        for word in row:
#Kata Utama Verba
#             if term1 in word:
#                 index=row.index(word) #index kata yang mengandung verba
#                 rulebox[2]=row[index][0]
#                 sentimen= row[index][2]
                
#                 cari1 = False
#                 temp1=index
#                 try:
#                     while cari1==False and temp1 <= len(row):
#                         temp1=temp1+1
#                         if term2 in row[temp1]: #index 1 kata setelah verba adj
#                             rulebox[3]=row[temp1][0]
#                             sentimen1 = sentimen and row[temp1][2]
#                             rulebox[5] = sentimen1
#                             cari1=True
#                         elif term4 in row[temp1]: #index 1 kata setelah verba nomina
#                             rulebox[3]=row[temp1][0]
#                             sentimen1 = sentimen is row[temp1][2]
#                             rulebox[5] = sentimen1
#                             cari1=True
#                 except:
#                     rulebox[5]=sentimen
                    
#                 cari2=False
#                 temp2=temp1
#                 try:
#                     while cari2==False and temp2 <= len(row):
#                         temp2=temp2+1
#                         if term2 in row[temp2] : #index 2 kata setelah verba adj
#                             rulebox[4]=row[temp2][0]
#                             sentimen2 = sentimen1 and row[temp2][2]
#                             rulebox[5] = sentimen2
#                             cari2=True
#                         elif term4 in row[temp2]: #index 2 kata setelah verba nomina
#                             rulebox[4]=row[temp2][0]
#                             sentimen2 = (row[index][2] is row[temp2][2]) and row[index+1][2]
#                             rulebox[5] = sentimen2
#                             cari2=True
#                 except:
#                     if cari1==False :
#                         rulebox[5]=row[index][2]
#                     else:
#                         rulebox[5] = sentimen1
                    
#                 cari3=False
#                 temp3=index
#                 try: 
#                     while cari3==False and temp3 >= 0:
#                         temp3=temp3-1
#                         if term5 in row[temp3]: #index 1 kata sebelum verba prepo
#                             if cari1==False and cari2==False:
#                                 sentimen3 = sentimen is row[temp3][2]
#                             elif cari1==True and cari2==False:
#                                 sentimen3 = sentimen1 is row[temp3][2]
#                             else:
#                                 sentimen3 = sentimen2 is row[temp3][2]
#                             rulebox[1]=row[temp3][0]
#                             rulebox[5] = sentimen3
#                             cari3=True
                            
#                         elif term4 in row[temp3]: #index 1 kata sebelum verba nomina
#                             if cari1==False and cari2==False:
#                                 sentimen3 = row[index][2] and row[temp3][2]
#                             elif cari1==True and cari2==False:
#                                 sentimen3 = sentimen1 and row[temp3][2]
#                             else:
#                                 sentimen3 = sentimen2 and row[temp3][2]
#                             rulebox[1]=row[temp3][0]
#                             rulebox[5] = sentimen3
#                             cari3=True
                            
#                 except:
#                     if cari1==False :
#                         rulebox[5]=row[index][2]
#                     else:
#                         rulebox[5] = sentimen2
                
#                 cari4=False
#                 temp4=temp3-1
#                 try:
#                     while cari4==False and temp4 >= 0:
#                         if term4 in row[temp4]: #index 2 kata sebelum verba nomina
#                             if cari1==False and cari2==False:
#                                 sentimen4 = sentimen is (row[temp3+1][2] is row[temp4][2])
#                             elif cari1==True and cari2==False:
#                                 sentimen4 = sentimen1 is (row[temp3+1][2] is row[temp4][2])
#                             else:
#                                 sentimen4 = sentimen2 is (row[temp3+1][2] is row[temp4][2])
#                             rulebox[0]=row[temp4][0]
#                             rulebox[5]=sentimen4
#                             cari4=True
#                         temp4=temp4-1
                                        
#                 except:
#                     rulebox[5]=sentimen3
# Ending kata utama verba

# # Kata Utama Adjektif
#             if term2 in word:
#                 index=row.index(word) #index kata yang mengandung adjektiva
#                 rulebox[2]=row[index][0]
#                 sentimen=row[index][2]
                
#                 cari1 = False
#                 temp1=index-1
#                 try:
#                     while cari1==False and temp1>= 0:
#                         if term4 in row[temp1] : #index 1 kata sebelum adjektiva nomina
#                             rulebox[1]=row[temp1][0] #bermasalah
#                             sentimen1 = sentimen and row[temp1][2]
#                             print("Oke")
#                             rulebox[5] = sentimen1
#                             cari1=True
#                         elif term5 in row[temp1] : #index 1 kata sebelum adjektiva preposisi
#                             rulebox[1]=row[temp1][0]
#                             sentimen1 = sentimen is row[temp1][2]
#                             rulebox[5] = sentimen1
#                             cari1=True
#                         temp1=temp1-1
                        
#                 except:
#                     rulebox[5]=sentimen
                    
#                 cari2=False
#                 temp2=temp1
#                 try:
#                     while cari2==False and temp2 >= 0:
#                         if term4 in row[temp2]: #index 2 kata sebelum adjektiva nomina
#                             rulebox[0]=row[temp2][0]
#                             sentimen2 = sentimen1 is row[temp2][2]
#                             rulebox[5] = sentimen2
#                             cari2=True
#                         temp2=temp2-1
#                 except:
#                     if cari1==False :
#                         rulebox[5]=sentimen
#                     else:
#                         print("Oke")
#                         rulebox[5] = sentimen1
                    
#                 cari3=False
#                 temp3=index+1
#                 try:
#                     while cari3==False and temp3 <= len(row):
#                         if term1 in row[temp3] : #index 1 kata sesudah adjektiva verba
#                             if cari1==False and cari2==False:
#                                 sentimen3 = sentimen and row[temp3][2]
#                             elif cari1==True and cari2==False and row[temp1][1] == "Preposisi":
#                                 sentimen3 = sentimen1 is row[temp3][2]
#                             elif cari1==True and cari2==False and row[temp1][1] == "Nomina":
#                                 sentimen3 = sentimen1 and row[temp3][2]
#                             else:
#                                 sentimen3 = sentimen2 is row[temp3][2]
#                             rulebox[3]=row[temp3][0]
#                             rulebox[5] = sentimen3
#                             cari3=True
#                         temp3=temp3+1
                        
#                 except:
#                     if cari1==False :
#                         rulebox[5]=sentimen
#                     else:
#                         rulebox[5] = sentimen2
                
#                 cari4=False
#                 temp4=temp3
#                 try:
#                     while cari4==False and temp4 <= len(row):
#                         if term4 in row[temp4]: #index 2 kata sesudah adjektiva nomina
#                             rulebox[4]=row[temp4][0]
#                             sentimen4 = sentimen3 is row[temp4][2]
#                             rulebox[5]=sentimen4
#                             cari4=True
#                         temp4=temp4+1 
                                         
#                 except:
#                     rulebox[5]=sentimen3
# End Kata Utama Adjektif
                    
#Kata utama verba-di
            cari = False
            while cari == False:
                if term3 in word:
                    index=row.index(word) #index kata yang mengandung verba_di
                    rulebox[2]=row[index][0]
                    sentimen=row[index][2]
                
                    cari1 = False
                    temp1=index-1
                    try:
                        while cari1==False and temp1>= 0:
                            if term4 in row[temp1] : #index 1 kata sebelum vebra-di nomina
                                rulebox[1]=row[temp1][0] 
                                sentimen1 = sentimen is row[temp1][2]
                                rulebox[5] = sentimen1
                                cari1=True
                            elif term5 in row[temp1] : #index 1 kata sebelum vebra-di preposisi
                                rulebox[1]=row[temp1][0]
                                sentimen1 = sentimen is row[temp1][2]
                                rulebox[5] = sentimen1
                                cari1=True
                            temp1=temp1-1
                        
                    except:
                        rulebox[5]=sentimen
                    
                    cari2=False
                    temp2=temp1
                    try:
                        while cari2==False and temp2 >= 0:
                            if term4 in row[temp2]: #index 2 kata sebelum vebra-di nomina
                                rulebox[0]=row[temp2][0]
                                sentimen2 = sentimen1 is row[temp2][2]
                                rulebox[5] = sentimen2
                                cari2=True
                            temp2=temp2-1
                    except:
                        if cari1==False :
                            rulebox[5]=sentimen
                        else:
                            rulebox[5] = sentimen1
                    
                    cari3=False
                    temp3=index+1
                    try:
                        while cari3==False and temp3 <= len(row):
                            if term2 in row[temp3] : #index 1 kata sesudah vebra-di adjektiva
                                if cari1==False and cari2==False:
                                    sentimen3 = sentimen and row[temp3][2]
                                elif cari1==True and cari2==False and row[temp1][1] == "Preposisi":
                                    sentimen3 = not (sentimen1 or row[temp3][2])
                                elif cari1==True and cari2==False and row[temp1][1] == "Nomina":
                                    sentimen3 = sentimen1 and row[temp3][2]
                                else:
                                    sentimen3 = not (sentimen2 or row[temp3][2])
                                rulebox[3]=row[temp3][0]
                                rulebox[5] = sentimen3
                                cari3=True
                                
                            elif term4 in row[temp3] : #index 1 kata sesudah vebra-di nomina
                                if cari1==False and cari2==False:
                                    sentimen3 = sentimen and row[temp3][2]
                                elif cari1==True and cari2==False and row[temp1][1] == "Preposisi":
                                    sentimen3 = (not sentimen1) is (not row[temp3][2])
                                elif cari1==True and cari2==False and row[temp1][1] == "Nomina":
                                    sentimen3 = sentimen1 and row[temp3][2]
                                else:
                                    sentimen3 = (not sentimen2) is (not row[temp3][2])
                                rulebox[3]=row[temp3][0]
                                rulebox[5] = sentimen3
                                cari3=True
                            temp3=temp3+1
                        
                    except:
                        if cari1==False :
                            rulebox[5]=sentimen
                        else:
                            rulebox[5] = sentimen2
                
                    cari4=False
                    temp4=temp3
                    try:
                        while cari4==False and temp4 <= len(row):
                            if term4 in row[temp4] : #index 2 kata sesudah verba-di nomina
                                if cari1==False:
                                    sentimen4 = sentimen3 and row[temp4][2]
                                else:
                                    sentimen4 = sentimen3 is row[temp4][2]
                                rulebox[4]=row[temp4][0]
                                rulebox[5]=sentimen4
                                cari4=True
                            elif term2 in row[temp4] : #index 2 kata sesudah verba-di adjektiva
                                if cari1==False:
                                    sentimen4 = sentimen3 and row[temp4][2]
                                else:
                                    sentimen4 = sentimen3 and row[temp4][2]
                                rulebox[4]=row[temp4][0]
                                rulebox[5]=sentimen4
                                cari4=True
                       
                            temp4=temp4+1 
                                         
                    except:
                        rulebox[5]=sentimen3

                    
        hasil_rulebox.append(rulebox)
    print(hasil_rulebox)

In [10]:
#Read the tweets one by one and process it
fp = open('data/sampleTweetsID.csv', 'r')
line = fp.readline()
st = open('data/feature_list/konjungsi.csv', 'r')
stopWords = getStopWordList('data/feature_list/konjungsi.csv')
result_list = []

In [11]:
while line:
    processedTweet = processTweet(line)
    featureVector = getFeatureVector(processedTweet)
    replaceAbb = translator(featureVector)
    temp = posTagging(replaceAbb)
    result_list.append(temp)
    line = fp.readline()
#end loop

[['korupsi', 'Nomina', False], ['sangat', 'Preposisi', True], ['buruk', 'Adjektiva', False], ['memajukan', 'Verba', True], ['indonesia', 'Nomina', True]]
 
[['korupsi', 'Nomina', False], ['sangat', 'Preposisi', True], ['buruk', 'Adjektiva', False], ['menciptakan', 'Verba', True], ['kehancuran', 'Nomina', False]]
 
[['korupsi', 'Nomina', False], ['sangat', 'Preposisi', True], ['buruk', 'Adjektiva', False], ['mengganggu', 'Verba', False], ['kedamaian', 'Nomina', True]]
 
[['korupsi', 'Nomina', False], ['sangat', 'Preposisi', True], ['buruk', 'Adjektiva', False], ['menghancurkan', 'Verba', False], ['kejahatan', 'Nomina', False]]
 


In [ ]:
#convert list to pickle/excel
# pickle.dump(result_list,open("result_list.p" , "wb"))
RuleDes(result_list)
fp.close()

In [1]:
print("hello")

hello
